In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import glob
import re
import pandas as pd
from functools import reduce
from klepto.archives import dir_archive

import numpy as np
from tqdm.auto import tqdm
import coffea.processor as processor
from coffea.processor.accumulator import AccumulatorABC
from coffea.analysis_objects import JaggedCandidateArray
from coffea.btag_tools import BTagScaleFactor
from coffea import hist
import pandas as pd
import uproot_methods
import uproot
import awkward
import copy

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from Tools.config_helpers import *
from Tools.helpers import mergeArray, mt, get_scheduler_address

from Tools.objects import Collections
from Tools.cutflow import Cutflow

# This just tells matplotlib not to open any
# interactive windows.
matplotlib.use('Agg')

In [2]:
def pad_and_flatten(val): 
    try:
        return val.pad(1, clip=True).fillna(0.).flatten()#.reshape(-1, 1)
    except AttributeError:
        return val.flatten()

#os.environ['KERAS_BACKEND'] = 'theano'
#from keras.models import load_model

In [3]:
import sys
sys.setrecursionlimit(10000)
print(sys.getrecursionlimit())

10000


In [4]:
from Tools.WH_objects import *
from Tools.WH_scalefactors import LeptonSF
from Tools.WH_deepAK8 import getWTagSF

In [5]:
#global variables... to avoid making bugs!!!
processesList = ['DYJets', 'TTJets', 'ttW', 'ttZ', 'Data']
linesList= ['triggers', 'filters',  'dimuon = 1', '70 < dimuon mass < 110']

plotDir = '/home/users/mbryson/public_html/dump/WH/dilep2016/'
year = 2016

In [21]:
class analysisProcessor(processor.ProcessorABC):
    """Processor used for running the analysis"""
    def __init__(self):
        
        ## load b-tag SFs
        #self.btag_sf = BTagScaleFactor(os.path.expandvars("$TWHOME/data/DeepCSV_102XSF_V1.btag.csv.gz", "reshape")

        ## load the NN
        #self.model = load_model('../ML/data/training.h5')
        #self.stds  = pd.read_json('../ML/data/stds.json').squeeze()
        #self.means = pd.read_json('../ML/data/means.json').squeeze()
        
        # we can use a large number of bins and rebin later
        dataset_axis        = hist.Cat("dataset",   "Primary dataset")
        pt_axis             = hist.Bin("pt",        r"$p_{T}$ (GeV)", 1000, 0, 1000)
        p_axis              = hist.Bin("p",         r"$p$ (GeV)", 1000, 0, 2500)
        ht_axis             = hist.Bin("ht",        r"$H_{T}$ (GeV)", 500, 0, 5000)
        mass_axis           = hist.Bin("mass",      r"Mass (GeV)", 1000, 0, 2000)
        eta_axis            = hist.Bin("eta",       r"$\eta$", 60, -5.5, 5.5)
        phi_axis            = hist.Bin("phi",       r"$\phi$", 40, -4, 4)
        delta_axis          = hist.Bin("delta",     r"$\delta$", 100,0,10 )
        multiplicity_axis   = hist.Bin("multiplicity",         r"N", 20, -0.5, 19.5)
        norm_axis           = hist.Bin("norm",         r"N", 25, 0, 1)

        self._accumulator = processor.dict_accumulator({
            "met":                     hist.Hist("Counts", dataset_axis, pt_axis),
            "leadmu_pt":               hist.Hist("Counts", dataset_axis, pt_axis),
            "leadmu_eta":              hist.Hist("Counts", dataset_axis, eta_axis),
            "leadmu_phi":              hist.Hist("Counts", dataset_axis, phi_axis),
            "subleadmu_pt":            hist.Hist("Counts", dataset_axis, pt_axis),
            "subleadmu_eta":           hist.Hist("Counts", dataset_axis, eta_axis),
            "subleadmu_phi":           hist.Hist("Counts", dataset_axis, phi_axis),
            "mm_deltaPhi":             hist.Hist("Counts", dataset_axis, delta_axis),
            "mm_deltaR":               hist.Hist("Counts", dataset_axis, delta_axis),
            "mm_mass":                 hist.Hist("Counts", dataset_axis, mass_axis),
            "mm_pt":                   hist.Hist("Counts", dataset_axis, pt_axis),
            "mm_eta":                  hist.Hist("Counts", dataset_axis, eta_axis),
            "mm_phi":                  hist.Hist("Counts", dataset_axis, phi_axis),
            "N_AK4" :                  hist.Hist("Counts", dataset_axis, multiplicity_axis),
            "N_AK8" :                  hist.Hist("Counts", dataset_axis, multiplicity_axis),
            "leadAK4_pt" :             hist.Hist("Counts", dataset_axis, pt_axis),            
            "leadAK4_eta" :            hist.Hist("Counts", dataset_axis, eta_axis),
            "leadAK4_phi" :            hist.Hist("Counts", dataset_axis, phi_axis),
            "subleadAK4_pt" :          hist.Hist("Counts", dataset_axis, pt_axis),            
            "subleadAK4_eta" :         hist.Hist("Counts", dataset_axis, eta_axis),
            "subleadAK4_phi" :         hist.Hist("Counts", dataset_axis, phi_axis),
            "leadAK8_pt" :             hist.Hist("Counts", dataset_axis, pt_axis),            
            "leadAK8_eta" :            hist.Hist("Counts", dataset_axis, eta_axis),            
            "leadAK8_phi" :            hist.Hist("Counts", dataset_axis, phi_axis), 
            "subleadAK8_pt" :          hist.Hist("Counts", dataset_axis, pt_axis),            
            "subleadAK8_eta" :         hist.Hist("Counts", dataset_axis, eta_axis),            
            "subleadAK8_phi" :         hist.Hist("Counts", dataset_axis, phi_axis),
            #"MET_pt_baseline" :          hist.Hist("Counts", dataset_axis, pt_axis),
            #"HT_baseline" :              hist.Hist("Counts", dataset_axis, ht_axis),
            #"mtb_min_baseline" :         hist.Hist("Counts", dataset_axis, mass_axis),
            #"MET_pt" :          hist.Hist("Counts", dataset_axis, pt_axis),
            #"HT" :              hist.Hist("Counts", dataset_axis, ht_axis),
            #"mtb_min" :         hist.Hist("Counts", dataset_axis, mass_axis),
            #"MET_pt_CR" :       hist.Hist("Counts", dataset_axis, pt_axis),
            #"HT_CR" :           hist.Hist("Counts", dataset_axis, ht_axis),
            #"mtb_min_CR" :      hist.Hist("Counts", dataset_axis, mass_axis),
            #"lead_AK8_pt" :     hist.Hist("Counts", dataset_axis, pt_axis),
            #"W_pt" :            hist.Hist("Counts", dataset_axis, pt_axis),
            #"H_pt" :            hist.Hist("Counts", dataset_axis, pt_axis),
            #"W_eta" :           hist.Hist("Counts", dataset_axis, eta_axis),
            #"H_eta" :           hist.Hist("Counts", dataset_axis, eta_axis),
            
#             "met_CR":           hist.Hist("Counts", dataset_axis, pt_axis),
#             "met_Higgs_CR":     hist.Hist("Counts", dataset_axis, pt_axis),
#             "met_W_CR":         hist.Hist("Counts", dataset_axis, pt_axis),
#             "met_Higgs_W_CR":   hist.Hist("Counts", dataset_axis, pt_axis),
            
#             "ht_CR":            hist.Hist("Counts", dataset_axis, ht_axis),
#             "ht_Higgs_CR":      hist.Hist("Counts", dataset_axis, ht_axis),
#             "ht_W_CR":          hist.Hist("Counts", dataset_axis, ht_axis),
#             "ht_Higgs_W_CR":    hist.Hist("Counts", dataset_axis, ht_axis),
            
#             "N_AK8_CR" :        hist.Hist("Counts", dataset_axis, multiplicity_axis),
#             "W_pt_CR" :         hist.Hist("Counts", dataset_axis, pt_axis),
#             "H_pt_CR" :         hist.Hist("Counts", dataset_axis, pt_axis),
#             "W_eta_CR" :        hist.Hist("Counts", dataset_axis, eta_axis),
#             "H_eta_CR" :        hist.Hist("Counts", dataset_axis, eta_axis),
            
#             "N_AK8_Higgs_CR" :  hist.Hist("Counts", dataset_axis, multiplicity_axis),
#             "W_pt_Higgs_CR" :   hist.Hist("Counts", dataset_axis, pt_axis),
#             "H_pt_Higgs_CR" :   hist.Hist("Counts", dataset_axis, pt_axis),
#             "W_eta_Higgs_CR" :  hist.Hist("Counts", dataset_axis, eta_axis),
#             "H_eta_Higgs_CR" :  hist.Hist("Counts", dataset_axis, eta_axis),
            
#             "N_AK8_W_CR" :      hist.Hist("Counts", dataset_axis, multiplicity_axis),
#             "W_pt_W_CR" :       hist.Hist("Counts", dataset_axis, pt_axis),
#             "H_pt_W_CR" :       hist.Hist("Counts", dataset_axis, pt_axis),
#             "W_eta_W_CR" :      hist.Hist("Counts", dataset_axis, eta_axis),
#             "H_eta_W_CR" :      hist.Hist("Counts", dataset_axis, eta_axis),
            
#             "N_AK8_Higgs_W_CR" :hist.Hist("Counts", dataset_axis, multiplicity_axis),
#             "W_pt_Higgs_W_CR" : hist.Hist("Counts", dataset_axis, pt_axis),
#             "H_pt_Higgs_W_CR" : hist.Hist("Counts", dataset_axis, pt_axis),
#             "W_eta_Higgs_W_CR" :hist.Hist("Counts", dataset_axis, eta_axis),
#             "H_eta_Higgs_W_CR" :hist.Hist("Counts", dataset_axis, eta_axis),

            #"N_b" :             hist.Hist("Counts", dataset_axis, multiplicity_axis),

            #"N_H" :             hist.Hist("Counts", dataset_axis, multiplicity_axis),
            #"N_W" :             hist.Hist("Counts", dataset_axis, multiplicity_axis),
            
            #"WH_deltaPhi":      hist.Hist("Counts", dataset_axis, delta_axis),
            #"WH_deltaR":        hist.Hist("Counts", dataset_axis, delta_axis),

            #"min_dphiJetMet4":  hist.Hist("Counts", dataset_axis, delta_axis),
            #"dphiDiJet":        hist.Hist("Counts", dataset_axis, delta_axis),
            #"dphiDiFatJet":     hist.Hist("Counts", dataset_axis, delta_axis),
            
#             'mC750_l1':         processor.defaultdict_accumulator(int),
#             'WJets':            processor.defaultdict_accumulator(int),
#             'QCD':              processor.defaultdict_accumulator(int),
            'TTJets':           processor.defaultdict_accumulator(int),
            'DYJets':           processor.defaultdict_accumulator(int),
#             'ZNuNu':            processor.defaultdict_accumulator(int),
#             'ST':               processor.defaultdict_accumulator(int),
#             'ST_tW':            processor.defaultdict_accumulator(int),
#             'ST_tChannel':      processor.defaultdict_accumulator(int),
#             'ST_sChannel':      processor.defaultdict_accumulator(int),
            'ttW':              processor.defaultdict_accumulator(int),
            'ttZ':              processor.defaultdict_accumulator(int),
#             'WW':               processor.defaultdict_accumulator(int),
#             'WZ/ZZ':            processor.defaultdict_accumulator(int),
#             'LL':               processor.defaultdict_accumulator(int),
            'Data':             processor.defaultdict_accumulator(int),
            'totalEvents':      processor.defaultdict_accumulator(int),
#             'test1':            processor.defaultdict_accumulator(float),
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        """
        Processing function. This is where the actual analysis happens.
        """
        output = self.accumulator.identity()
        dataset = df["dataset"]
        cfg = loadConfig()
        
        ############## MET ##############
        met = JaggedCandidateArray.candidatesfromcounts(
            (df['MET_pt']>=0)*1,
            pt = df["MET_pt"],
            eta = df["MET_phi"]*0,
            phi = df["MET_phi"],
            mass = df["MET_phi"]*0
        )
        
        
        ############## LOAD OBJECTS ############## 
        
        muon     = getMuons(df, WP='veto')
        electron = getElectrons(df, WP='veto')
        tau      = getTaus(df)
        isotrack = getIsoTracks(df)
        fatjet   = getFatJets(df)
        jet      = getJets(df)
        
        triggers = getDimuonTriggers(df, year=year, dataset=dataset)
        filters  = getFilters(df, year=year, dataset=dataset)
        
#         sf = LeptonSF(year=year)
#         leptonSF = sf.get(electron, muon)

        ############## LEPTONS ############## 
    
        high_pt_e = electron[electron.pt.argsort(ascending=False)][:,:2]
        ee = high_pt_e.choose(2)
        
        high_pt_m = muon[muon.pt.argsort(ascending=False)][:,:2]
        mm = high_pt_m.choose(2)
        newmet = mm.cross(met)

        
        mm_deltaPhi = np.arccos(np.cos(mm.i0.phi-mm.i1.phi))
        mm_deltaR = mm.i0.p4.delta_r(mm.i1.p4)
        
        leadm = high_pt_m[high_pt_m.pt.argmax()]
        subleadm = high_pt_m[high_pt_m.pt.argmin()]
        
        #selection for two muons
        dimuonsel = (muon.counts == 2)
        
        ############## FATJETS ##############

        
        high_pt_fatjet = fatjet[fatjet.pt.argsort(ascending=False)][:,:2]
        leadfatjet = high_pt_fatjet[high_pt_fatjet.pt.argmax()]
        subleadfatjet = high_pt_fatjet[high_pt_fatjet.pt.argmin()]
        
        leadingFatJets = fatjet[:,:2]
        difatjet = leadingFatJets.choose(2)
        dphiDiFatJet = np.arccos(np.cos(difatjet.i0.phi-difatjet.i1.phi))
        
        htag = fatjet[((fatjet.pt > 200) & (fatjet.deepTagMD_HbbvsQCD > 0.8365))]
        htag_hard = fatjet[((fatjet.pt > 300) & (fatjet.deepTagMD_HbbvsQCD > 0.8365))]
        
        lead_htag = htag[htag.pt.argmax()]
        
        wtag = fatjet[((fatjet.pt > 200) & (fatjet.deepTagMD_HbbvsQCD < 0.8365) & (fatjet.deepTag_WvsQCD > 0.918))]
        wtag_hard = fatjet[((fatjet.pt > 300) & (fatjet.deepTagMD_HbbvsQCD < 0.8365) & (fatjet.deepTag_WvsQCD > 0.918))]
        
        lead_wtag = wtag[wtag.pt.argmax()]
        
        wh = lead_htag.cross(lead_wtag)
        wh_deltaPhi = np.arccos(wh.i0.phi - wh.i1.phi)
        wh_deltaR = wh.i0.p4.delta_r(wh.i1.p4)
        
        ############## JETS ##############

        skimjet   = jet[(jet.pt>30) & (abs(jet.eta)<2.4)]
        jet       = jet[(jet.pt>30) & (jet.jetId>1) & (abs(jet.eta)<2.4)]
        jet       = jet[~jet.match(muon, deltaRCut=0.4)] # remove jets that overlap with muons
        jet       = jet[~jet.match(electron, deltaRCut=0.4)] # remove jets that overlap with electrons
        jet       = jet[~jet.match(fatjet, deltaRCut=1.2)] # remove AK4 jets that overlap with AK8 jets
        jet       = jet[jet.pt.argsort(ascending=False)] # sort the jets
        btag      = jet[(jet.btagDeepB>0.4184)]
        light     = jet[(jet.btagDeepB<0.4184)]
        extrajet  = jet[~jet.match(fatjet, deltaRCut=0.8)] 
        
        ## Get leading jets
        
        high_pt_jet = jet[jet.pt.argsort(ascending=False)][:,:2]
        leadjet = high_pt_jet[high_pt_jet.pt.argmax()]
        subleadjet = high_pt_jet[high_pt_jet.pt.argmin()]
        
        ## Get the leading b-jets
        high_score_btag = jet[jet.btagDeepB.argsort(ascending=False)][:,:2]
        
        leading_jet    = jet[jet.pt.argmax()]
        leading_b      = btag[btag.pt.argmax()]
        
        bb = high_score_btag.choose(2)
        bb_deltaPhi = np.arccos(np.cos(bb.i0.phi-bb.i1.phi))
        bb_deltaR = bb.i0.p4.delta_r(bb.i1.p4)
#        print(met[dimuonsel].pt)# + btag[dimuonsel].pt)


        mtb = mt(btag[dimuonsel].pt, btag[dimuonsel].phi, newmet[dimuonsel].pt.flatten(), newmet[dimuonsel].phi.flatten())
        min_mtb = mtb.min()
        mth = mt(htag[dimuonsel].pt, htag[dimuonsel].phi, newmet[dimuonsel].pt.flatten(), newmet[dimuonsel].phi.flatten())

        ############## OTHER VARIABLES ##############
        
        ht = jet.pt.sum()
        
        min_dphiJetMet4 = np.arccos(np.cos(jet[dimuonsel][:,:4].phi-newmet[dimuonsel].phi.flatten())).min()
        
        leadingJets = jet[:,:2]
        dijet = leadingJets.choose(2)
        dphiDiJet = np.arccos(np.cos(dijet.i0.phi-dijet.i1.phi))
        
        min_dphiFatJetMet4 = np.arccos(np.cos(fatjet[dimuonsel][:,:4].phi-newmet[dimuonsel].phi.flatten())).min()
       
        ############## SELECTIONS ##############
        
        dilep_sel = ((electron.counts+muon.counts)==2)
        dilep_sf_sel = dilep_sel & ((electron.counts ==2 )|(muon.counts == 2))
        dilep_of_sel = dilep_sel & ((electron.counts ==1 )|(muon.counts == 1))
        
        dimuon_sel = dilep_sel & (muon.counts == 2)
        
        ak4_sel = (jet.counts > 1)
        ak8_sel = (fatjet.counts > 0)
        
        dimuonmass_sel = (abs(mm.mass-90) < 20).any()
        dimuonpt_sel = (mm.pt > 200).any()
        leadmuon_sel = (leadm.pt >= 20).any()

        ############## CUTFLOW ##############
        
        output['totalEvents']['all'] += len(df['weight'])
        processes = processesList
        weight = np.ones(len(df['weight'])) if dataset=='Data' else df['weight']
        lumi = 1 if dataset=='Data' else 36
        fullweight = weight * lumi
        
        cutflow = Cutflow(output, df, cfg, processes, weight=fullweight)
        
        cutflow.addRow( 'triggers',   (triggers) )         
        cutflow.addRow( 'filters',   (filters) ) 
        cutflow.addRow('dimuon = 1', (dimuon_sel))
        cutflow.addRow('70 < dimuon mass < 110', (dimuonmass_sel))
#        cutflow.addRow( 'MET>250',     (newmet.pt>250).any() )
#         cutflow.addRow( 'N_fatjet>1',      (fatjet.counts>1) )
#         cutflow.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet4>0.5).any())
#         cutflow.addRow( 'dphiDiFatJet', (dphiDiFatJet<2.5).any() ) 
#         cutflow.addRow( 'htag > 0',   (htag_sel) )
#         cutflow.addRow( 'minmth>200',   (mth.min()>200).any() )
#         cutflow.addRow( '90 < h sd mass > 150',   (htag_sel) )
#         cutflow.addRow( 'njet veto',     (extrajet.counts<2))

#         cutflow.addRow('lead muon pt >= 20', (leadmuon_sel))
#        cutflow.addRow('dimuon pt > 200', (dimuonpt_sel))       
#         cutflow.addRow( 'stitch',   (stitchVar ==1) )
#         cutflow.addRow( 'skim',   ((met_pt>200) & (skimjet.counts>1)) )
#         cutflow.addRow( 'Exactly 1 e or mu',   ((electron.counts+muon.counts)==1) )
# 
        
        baseline = copy.deepcopy(cutflow.selection)
        

#         cutflow.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet4>0.5))
#         cutflow.addRow( 'dphiDiFatJet', (dphiDiFatJet<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets

#         cutflow.addRow( 'njet veto',     (jet.counts<2))

#         vetoQCD = copy.deepcopy(cutflow.selection)
        
#         cutflow.addRow( 'N_wtag>0',     (wtag_sel), cumulative=False)
        
#         wtag_selection = copy.deepcopy(cutflow.selection)
        
#         cutflow.addRow( 'N_htag>0',     (htag_sel), cumulative=False)

#         htag_selection = copy.deepcopy(cutflow.selection)
        
#         cutflow.addRow( 'N_htag>0, N_wtag>0',     (htag_sel & wtag_sel))

#         signal_selection = cutflow.selection
        
        ############## HISTOGRAMS ##############
        output['met'].fill(dataset=dataset,pt=newmet[baseline].pt.flatten(),weight=fullweight[baseline])
        output['mm_mass'].fill(dataset=dataset,mass=mm[baseline].mass.flatten(),weight=fullweight[baseline])
        output['mm_pt'].fill(dataset=dataset,pt=mm[baseline].pt.flatten(),weight=fullweight[baseline])
        output['mm_eta'].fill(dataset=dataset,eta=mm[baseline].eta.flatten(),weight=fullweight[baseline])
        output['mm_phi'].fill(dataset=dataset,phi=mm[baseline].phi.flatten(),weight=fullweight[baseline])
        output['leadmu_pt'].fill(dataset=dataset,pt=leadm[baseline].pt.flatten(),weight=fullweight[baseline])
        output['leadmu_eta'].fill(dataset=dataset,eta=leadm[baseline].eta.flatten(),weight=fullweight[baseline])
        output['leadmu_phi'].fill(dataset=dataset,phi=leadm[baseline].phi.flatten(),weight=fullweight[baseline])
        output['subleadmu_pt'].fill(dataset=dataset,pt=subleadm[baseline].pt.flatten(),weight=fullweight[baseline])
        output['subleadmu_eta'].fill(dataset=dataset,eta=subleadm[baseline].eta.flatten(),weight=fullweight[baseline])
        output['subleadmu_phi'].fill(dataset=dataset,phi=subleadm[baseline].phi.flatten(),weight=fullweight[baseline])
        output['mm_deltaPhi'].fill(dataset=dataset, delta=mm_deltaPhi[baseline].flatten(), weight=fullweight[baseline])
        output['mm_deltaR'].fill(dataset=dataset, delta=mm_deltaR[baseline].flatten(), weight=fullweight[baseline])
        output['N_AK4'].fill(dataset=dataset, multiplicity=jet[baseline].counts, weight=fullweight[baseline])
        output['N_AK8'].fill(dataset=dataset, multiplicity=fatjet[baseline].counts, weight=fullweight[baseline])       
        output['leadAK4_pt'].fill(dataset=dataset, pt=leadjet[baseline & ak4_sel].pt.flatten(), weight=fullweight[baseline & ak4_sel])
        output['leadAK4_eta'].fill(dataset=dataset, eta=leadjet[baseline & ak4_sel].eta.flatten(), weight=fullweight[baseline & ak4_sel])
        output['leadAK4_phi'].fill(dataset=dataset, phi=leadjet[baseline & ak4_sel].phi.flatten(), weight=fullweight[baseline & ak4_sel])
        output['subleadAK4_pt'].fill(dataset=dataset, pt=subleadjet[baseline & ak4_sel].pt.flatten(), weight=fullweight[baseline & ak4_sel])
        output['subleadAK4_eta'].fill(dataset=dataset, eta=subleadjet[baseline & ak4_sel].eta.flatten(), weight=fullweight[baseline & ak4_sel])
        output['subleadAK4_phi'].fill(dataset=dataset, phi=subleadjet[baseline & ak4_sel].phi.flatten(), weight=fullweight[baseline & ak4_sel])
        output['leadAK8_pt'].fill(dataset=dataset, pt=leadfatjet[baseline & ak8_sel].pt.flatten(), weight=fullweight[baseline & ak8_sel])       
        output['leadAK8_eta'].fill(dataset=dataset, eta=leadfatjet[baseline & ak8_sel].eta.flatten(), weight=fullweight[baseline & ak8_sel])       
        output['leadAK8_phi'].fill(dataset=dataset, phi=leadfatjet[baseline & ak8_sel].phi.flatten(), weight=fullweight[baseline & ak8_sel])       
        output['subleadAK8_pt'].fill(dataset=dataset, pt=subleadfatjet[baseline & ak8_sel].pt.flatten(), weight=fullweight[baseline & ak8_sel])       
        output['subleadAK8_eta'].fill(dataset=dataset, eta=subleadfatjet[baseline & ak8_sel].eta.flatten(), weight=fullweight[baseline & ak8_sel])       
        output['subleadAK8_phi'].fill(dataset=dataset, phi=subleadfatjet[baseline & ak8_sel].phi.flatten(), weight=fullweight[baseline & ak8_sel])       

        
        #         output['met_CR'].fill(dataset=dataset, pt=met_pt[vetoQCD].flatten(), weight=fullweight[vetoQCD])
#         output['met_W_CR'].fill(dataset=dataset, pt=met_pt[vetoQCD & wtag_sel].flatten(), weight=fullweight[vetoQCD & wtag_sel])
#         output['met_Higgs_CR'].fill(dataset=dataset, pt=met_pt[vetoQCD & htag_sel].flatten(), weight=fullweight[vetoQCD & htag_sel])
#         output['met_Higgs_W_CR'].fill(dataset=dataset, pt=met_pt[signal_selection].flatten(), weight=fullweight[signal_selection])

#         output['ht_CR'].fill(dataset=dataset, ht=ht[vetoQCD].flatten(), weight=fullweight[vetoQCD])
#         output['ht_W_CR'].fill(dataset=dataset, ht=ht[vetoQCD & wtag_sel].flatten(), weight=fullweight[vetoQCD & wtag_sel])
#         output['ht_Higgs_CR'].fill(dataset=dataset, ht=ht[vetoQCD & htag_sel].flatten(), weight=fullweight[vetoQCD & htag_sel])
#         output['ht_Higgs_W_CR'].fill(dataset=dataset, ht=ht[signal_selection].flatten(), weight=fullweight[signal_selection])
        
#         output['N_AK8_CR'].fill(dataset=dataset, multiplicity=fatjet[vetoQCD].counts, weight=fullweight[vetoQCD])
#         #output['W_pt_CR'].fill(dataset=dataset, pt=lead_wtag[vetoQCD].pt.flatten(), weight=fullweight[vetoQCD])
        #output['H_pt_CR'].fill(dataset=dataset, pt=lead_htag[vetoQCD].pt.flatten(), weight=fullweight[vetoQCD])
        #output['W_eta_CR'].fill(dataset=dataset, eta=lead_wtag[vetoQCD].eta.flatten(), weight=fullweight[vetoQCD])
        #output['H_eta_CR'].fill(dataset=dataset, eta=lead_htag[vetoQCD].eta.flatten(), weight=fullweight[vetoQCD])

#         output['N_AK8_W_CR'].fill(dataset=dataset, multiplicity=fatjet[vetoQCD & wtag_sel].counts, weight=fullweight[vetoQCD & wtag_sel])
#         output['W_pt_W_CR'].fill(dataset=dataset, pt=lead_wtag[vetoQCD & wtag_sel].pt.flatten(), weight=fullweight[vetoQCD & wtag_sel])
#         #output['H_pt_W_CR'].fill(dataset=dataset, pt=lead_htag[vetoQCD & wtag_sel].pt.flatten(), weight=fullweight[vetoQCD & wtag_sel])
        #output['W_eta_W_CR'].fill(dataset=dataset, eta=lead_wtag[vetoQCD & wtag_sel].eta.flatten(), weight=fullweight[vetoQCD & wtag_sel])
        #output['H_eta_W_CR'].fill(dataset=dataset, eta=lead_htag[vetoQCD & wtag_sel].eta.flatten(), weight=fullweight[vetoQCD & wtag_sel])

        #output['N_AK8_Higgs_CR'].fill(dataset=dataset, multiplicity=fatjet[vetoQCD & htag_sel].counts, weight=fullweight[vetoQCD & htag_sel])
        #output['W_pt_Higgs_CR'].fill(dataset=dataset, pt=lead_wtag[vetoQCD & htag_sel].pt.flatten(), weight=fullweight[vetoQCD & htag_sel])
        #output['H_pt_Higgs_CR'].fill(dataset=dataset, pt=lead_htag[vetoQCD & htag_sel].pt.flatten(), weight=fullweight[vetoQCD & htag_sel])
        #output['W_eta_Higgs_CR'].fill(dataset=dataset, eta=lead_wtag[vetoQCD & htag_sel].eta.flatten(), weight=fullweight[vetoQCD & htag_sel])
        #output['H_eta_Higgs_CR'].fill(dataset=dataset, eta=lead_htag[vetoQCD & htag_sel].eta.flatten(), weight=fullweight[vetoQCD & htag_sel])

#         output['N_AK8_Higgs_W_CR'].fill(dataset=dataset, multiplicity=fatjet[signal_selection].counts, weight=fullweight[signal_selection])
#         output['W_pt_Higgs_W_CR'].fill(dataset=dataset, pt=lead_wtag[signal_selection].pt.flatten(), weight=fullweight[signal_selection])
#         output['H_pt_Higgs_W_CR'].fill(dataset=dataset, pt=lead_htag[signal_selection].pt.flatten(), weight=fullweight[signal_selection])
#         output['W_eta_Higgs_W_CR'].fill(dataset=dataset, eta=lead_wtag[signal_selection].eta.flatten(), weight=fullweight[signal_selection])
#         output['H_eta_Higgs_W_CR'].fill(dataset=dataset, eta=lead_htag[signal_selection].eta.flatten(), weight=fullweight[signal_selection])

                
#         output['MET_pt_baseline'].fill(dataset=dataset, pt=met_pt[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
#         output['HT_baseline'].fill(dataset=dataset, ht=ht[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
#         output['mtb_min_baseline'].fill(dataset=dataset, mass=mtb[baseline].min().flatten(), weight=df['weight'][baseline]*cfg['lumi'])

#         output['MET_pt'].fill(dataset=dataset, pt=met_pt[vetoQCD].flatten(), weight=df['weight'][vetoQCD]*cfg['lumi'])
#         output['HT'].fill(dataset=dataset, ht=ht[vetoQCD].flatten(), weight=df['weight'][vetoQCD]*cfg['lumi'])
#         output['mtb_min'].fill(dataset=dataset, mass=mtb[vetoQCD].min().flatten(), weight=df['weight'][vetoQCD]*cfg['lumi'])
        


#         output['min_dphiJetMet4'].fill(dataset=dataset, delta=min_dphiJetMet4[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
#         output['dphiDiJet'].fill(dataset=dataset, delta=dphiDiJet[baseline].min().flatten(), weight=df['weight'][baseline]*cfg['lumi'])

#         ## Higgs and W pt
#         output['lead_AK8_pt'].fill(dataset=dataset, pt=fatjet[(baseline & (fatjet.counts>0))].pt.max().flatten(), weight=df['weight'][(baseline & (fatjet.counts>0))]*cfg['lumi'])
#         output['dphiDiFatJet'].fill(dataset=dataset, delta=dphiDiFatJet[(baseline & (fatjet.counts>1))].min().flatten(), weight=df['weight'][(baseline & (fatjet.counts>1))]*cfg['lumi'])

#         output['H_pt'].fill(dataset=dataset, pt=lead_htag[event_selection].pt.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['H_eta'].fill(dataset=dataset, eta=lead_htag[event_selection].eta.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

#         output['W_pt'].fill(dataset=dataset, pt=lead_wtag[event_selection].pt.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['W_eta'].fill(dataset=dataset, eta=lead_wtag[event_selection].eta.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

#         output['WH_deltaPhi'].fill(dataset=dataset, delta=wh_deltaPhi[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['WH_deltaR'].fill(dataset=dataset, delta=wh_deltaR[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

#         output['MET_pt_CR'].fill(dataset=dataset, pt=met_pt[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['HT_CR'].fill(dataset=dataset, ht=ht[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
#         output['mtb_min_CR'].fill(dataset=dataset, mass=mtb[event_selection].min().flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

        return output

    def postprocess(self, accumulator):
        return accumulator

In [22]:
runLocal = True

if not runLocal:
    # Get the scheduler from the dask_cluster notebook
    from dask.distributed import Client, progress

    c = Client('tcp://169.228.130.5:27879')

    ## for dask
    exe_args = {
        'client': c,
        #'savemetrics': True,
    }
    exe = processor.dask_executor
    
else:
    ## for local
    exe_args = {
        'workers': 16,
        'function_args': {'flatten': False}
    }
    exe = processor.futures_executor

if not runLocal:
    print(c)

In [23]:
overwrite = True
small = True

tag = 'v0.2.4'

from Tools.dilep_samples import * #fileset_2016, fileset_2016_small

if year == 2016:
    fileset_year = fileset_dilep_2016
    fileset_year_small = fileset_dilep_2016_small
elif year == 2017:
    fileset_year = fileset_2017
    fileset_year_small = fileset_dilep_2017_small
elif year == 2018:
    fileset_year = fileset_2018
    fileset_year_small = fileset_dilep_2018_small


fileset_dilep = {
    'DYJets': fileset_year['DY_HT'],
    'TTJets': fileset_year['TTJets'] + fileset_year['TTJets_ext'],
    'ttW': fileset_year['TTW'],
    'ttZ': fileset_year['TTZ'],
    'Data': fileset_year['DoubleMuon'],
}


fileset_dilep_small = {
    'DYJets': fileset_dilep['DYJets'][:2],
    'TTJets': fileset_dilep['TTJets'][:2],
    'ttW': fileset_dilep['ttW'][:2],
    'ttZ': fileset_dilep['ttZ'][:2],
    'Data': fileset_dilep['Data'][:2],
}

# load the config and the cache
cfg = loadConfig()

cacheName = 'Dilep_%s_small'%year if small else 'DiLep_%s'%year
print(cacheName)

# histograms
histograms = []

# initialize cache
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)
if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')

else:
    # Run the processor
    if small:
        fileset = fileset_dilep_small
        exe_args['workers'] = 4
    else:
        fileset = fileset_dilep
        exe_args['workers'] = 16
    
        
    output = processor.run_uproot_job(fileset,
                                      treename='Events',
                                      processor_instance=analysisProcessor(),
                                      executor=exe,
                                      executor_args=exe_args,
                                      #chunksize=250000,
                                      chunksize=100000,
                                     )
    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

Dilep_2016_small


HBox(children=(HTML(value='Processing'), FloatProgress(value=0.0, max=80.0), HTML(value='')))

/home/users/mbryson/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/mbryson/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/mbryson/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/mbryson/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [24]:
# Cutflow
from Tools.helpers import getCutFlowTable

processes = processesList
lines     = ['entry']
lines    += linesList
df        = getCutFlowTable(output, processes=processes, lines=lines, significantFigures=4)
df

,DYJets,TTJets,ttW,ttZ,Data
entry,1554000.0 +/- 1000.0,390400.0 +/- 200.0,2359.0 +/- 5.0,2586.0 +/- 4.0,269400.0 +/- 500.0
filters,1552000.0 +/- 1000.0,390000.0 +/- 200.0,2353.0 +/- 5.0,2580.0 +/- 4.0,234800.0 +/- 500.0
triggers,1554000.0 +/- 1000.0,390400.0 +/- 200.0,2359.0 +/- 5.0,2586.0 +/- 4.0,235300.0 +/- 500.0
dimuon = 1,770900.0 +/- 800.0,92420.0 +/- 110.0,407.4 +/- 1.9,490.9 +/- 1.9,229100.0 +/- 500.0
70 < dimuon mass < 110,714800.0 +/- 800.0,24090.0 +/- 60.0,96.82 +/- 0.92,343.1 +/- 1.6,185700.0 +/- 400.0


In [12]:
# Efficiencies
df = getCutFlowTable(output, processes=processes, lines=lines, significantFigures=3, absolute=False)
df

,DYJets,TTJets,ttW,ttZ,Data
entry,1.000,1.000,1.000,1.000,1.000
filters,0.999,0.999,0.998,0.998,0.998
triggers,0.486,0.429,0.419,0.374,0.873
dimuon = 1,0.971,0.514,0.384,0.481,0.976
70 < dimuon mass < 110,0.930,0.264,0.240,0.712,0.811


In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
from plots.helpers import *
bins = {
    'mm_mass_axis':        {'axis':'mass',   'overflow': 'over', 'bins': hist.Bin("mass",  r"Dimuon mass (GeV)", 25, 50, 150)},
    'mm_pt_axis':          {'axis':'pt',   'overflow': 'over', 'bins': hist.Bin("pt",  r"Dimuon $p_{T}$ (GeV)", 25, 0, 300)},
    'mm_eta_axis':         {'axis':'eta',   'overflow': 'over', 'bins': hist.Bin("eta",       r"Dimuon $\eta$", 30, -5.5, 5.5)},
    'mm_phi_axis':         {'axis':'phi',   'overflow': 'over', 'bins': hist.Bin("phi",       r"Dimuon $\phi$", 20, -4, 4)},
    'leadmu_pt_axis':      {'axis':'pt',   'overflow': 'over', 'bins': hist.Bin("pt",  r" Lead Muon $p_{T}$ (GeV)", 25, 0, 250)},
    'leadmu_eta_axis':     {'axis':'eta',   'overflow': 'over', 'bins': hist.Bin("eta",       r"Lead Muon $\eta$", 30, -5.5, 5.5)},
    'leadmu_phi_axis':     {'axis':'phi',   'overflow': 'over', 'bins': hist.Bin("phi",       r"Lead Muon $\phi$", 20, -4, 4)},
    'subleadmu_pt_axis':   {'axis':'pt',   'overflow': 'over', 'bins': hist.Bin("pt",  r" Sublead Muon $p_{T}$ (GeV)", 25, 0, 150)},
    'subleadmu_eta_axis':  {'axis':'eta',   'overflow': 'over', 'bins': hist.Bin("eta",       r"Sublead Muon $\eta$", 30, -5.5, 5.5)},
    'subleadmu_phi_axis':  {'axis':'phi',   'overflow': 'over', 'bins': hist.Bin("phi",       r"Sublead Muon $\phi$", 20, -4, 4)},
    'leadAK4_pt_axis':     {'axis':'pt',   'overflow': 'over', 'bins': hist.Bin("pt",  r" Lead AK4 $p_{T}$ (GeV)", 25, 0, 300)},
    'leadAK4_eta_axis':    {'axis':'eta',   'overflow': 'over', 'bins': hist.Bin("eta",       r"Lead AK4 $\eta$", 30, -5.5, 5.5)},
    'leadAK4_phi_axis':    {'axis':'phi',   'overflow': 'over', 'bins': hist.Bin("phi",       r"Lead AK4 $\phi$", 20, -4, 4)},
    'subleadAK4_pt_axis':  {'axis':'pt',   'overflow': 'over', 'bins': hist.Bin("pt",  r" Sublead AK4 $p_{T}$ (GeV)", 25, 0, 200)},
    'subleadAK4_eta_axis': {'axis':'eta',   'overflow': 'over', 'bins': hist.Bin("eta",       r"Sublead AK4 $\eta$", 30, -5.5, 5.5)},
    'subleadAK4_phi_axis': {'axis':'phi',   'overflow': 'over', 'bins': hist.Bin("phi",       r"Sublead AK4 $\phi$", 20, -4, 4)},
    'leadAK8_pt_axis':     {'axis':'pt',   'overflow': 'over', 'bins': hist.Bin("pt",  r" Lead AK8 $p_{T}$ (GeV)", 25, 150, 400)},
    'leadAK8_eta_axis':    {'axis':'eta',   'overflow': 'over', 'bins': hist.Bin("eta",       r"Lead AK8 $\eta$", 30, -5.5, 5.5)},
    'leadAK8_phi_axis':    {'axis':'phi',   'overflow': 'over', 'bins': hist.Bin("phi",       r"Lead AK8 $\phi$", 20, -4, 4)},
    'subleadAK8_pt_axis':  {'axis':'pt',   'overflow': 'over', 'bins': hist.Bin("pt",  r" Sublead AK8 $p_{T}$ (GeV)", 25, 150, 350)},
    'subleadAK8_eta_axis': {'axis':'eta',   'overflow': 'over', 'bins': hist.Bin("eta",       r"Sublead AK8 $\eta$", 30, -5.5, 5.5)},
    'subleadAK8_phi_axis': {'axis':'phi',   'overflow': 'over', 'bins': hist.Bin("phi",       r"Sublead AK8 $\phi$", 20, -4, 4)},
    'N_AK4_axis':          {'axis': 'multiplicity',  'overflow':'over',  'bins': hist.Bin('multiplicity', r'$N_{AK4 jet}$', 6, -0.5, 5.5)},
    'N_AK8_axis':          {'axis': 'multiplicity',  'overflow':'over',  'bins': hist.Bin('multiplicity', r'$N_{AK8 jet}$', 5, -0.5, 4.5)},
    'mm_deltaPhi_axis':    {'axis': 'delta',          'overflow':'over',  'bins': hist.Bin('delta', r'Dimuon $\Delta \phi $', 30, 0, 3)},
    'mm_deltaR_axis':      {'axis': 'delta',          'overflow':'over',  'bins': hist.Bin('delta', r'Dimuon $\Delta R $', 10, 0, 5)},
   }

In [ ]:
#Plotting aesthetics

lineopts = {
    'color': 'r',
    'linewidth': '3'}

data_err_opts = {
    'linestyle': 'none',
    'marker': '_',
    'markersize': 10.,
    'color': 'r',
    'elinewidth': 1}

data_err_opts_rat = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'k',
    'elinewidth': 1}

fillopts2 = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#989C94'),('#6A0136'),('#FF5714'),('#FFCA3A')]  
}

In [ ]:
from plots.helpers import *

def saveFig( fig, ax, rax, path, name, scale='linear', shape=False, y_max=-1 ):
    outdir = os.path.join(path,scale)
    finalizePlotDir(outdir)
    ax.set_yscale(scale)
    ax.set_ylabel('Events')

    if scale == 'linear':
        if y_max<0: #or True:
            pass
        else:
            ax.set_ylim(0, 1 if shape else 1.2*y_max)
    else:
        if y_max<0 and not shape:
            pass
        else:
            ax.set_ylim(0.000005 if shape else 0.05, 3 if shape else 300*y_max)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = []
    for handle, label in zip(handles, labels):
        #print (handle, label)
        try:
            new_labels.append(my_labels[label])
            if not label=='pseudodata':
                handle.set_color(colors[label])
        except:
            pass

    if rax:
        plt.subplots_adjust(hspace=0)
        rax.set_ylabel('Obs./Pred.')
        rax.set_ylim(0,2)

    ax.legend(title='',ncol=2,handles=handles, labels=new_labels, frameon=False)

    fig.text(0., 0.995, '$\\bf{CMS}$', fontsize=20,  horizontalalignment='left', verticalalignment='bottom', transform=ax.transAxes )
    fig.text(0.15, 1., '$\\it{Simulation}$', fontsize=14, horizontalalignment='left', verticalalignment='bottom', transform=ax.transAxes )
    fig.text(0.8, 1., '13 TeV', fontsize=14, horizontalalignment='left', verticalalignment='bottom', transform=ax.transAxes )

    fig.savefig(os.path.join(outdir, "{}.pdf".format(name)))
    fig.savefig(os.path.join(outdir, "{}.png".format(name)))
    #ax.clear()

In [ ]:
#Histos I want to save
histos = [["mm_mass", "mm_mass_axis"],
        ["mm_pt", "mm_pt_axis"],
        ["mm_eta", "mm_eta_axis"],
        ["mm_phi", "mm_phi_axis"],
        ["leadmu_pt", "leadmu_pt_axis"],
        ["leadmu_eta", "leadmu_eta_axis"],
        ["leadmu_phi", "leadmu_phi_axis"],
        ["subleadmu_pt", "subleadmu_pt_axis"],
        ["subleadmu_eta", "subleadmu_eta_axis"],
        ["subleadmu_phi", "subleadmu_phi_axis"],
        ["leadAK4_pt", "leadAK4_pt_axis"],
        ["leadAK4_eta", "leadAK4_eta_axis"],
        ["leadAK4_phi", "leadAK4_phi_axis"],
        ["subleadAK4_pt", "subleadAK4_pt_axis"],
        ["subleadAK4_eta", "subleadAK4_eta_axis"],
        ["subleadAK4_phi", "subleadAK4_phi_axis"],
        ["leadAK8_pt", "leadAK8_pt_axis"],
        ["leadAK8_eta", "leadAK8_eta_axis"],
        ["leadAK8_phi", "leadAK8_phi_axis"],
        ["subleadAK8_pt", "subleadAK8_pt_axis"],
        ["subleadAK8_eta", "subleadAK8_eta_axis"],
        ["subleadAK8_phi", "subleadAK8_phi_axis"],
        ["N_AK4", "N_AK4_axis"],
        ["N_AK8", "N_AK8_axis"],
        ["mm_deltaPhi", "mm_deltaPhi_axis"],
        ["mm_deltaR", "mm_deltaR_axis"]]

In [ ]:
# Make some of the plots


finalizePlotDir(plotDir)


for plot in histos:

    name = plot[0]
    binName = plot[1]
    histogram = output[name]

    axis = bins[binName]['axis']
    histogram = histogram.rebin(axis, bins[binName]['bins'])

    y_max = histogram.sum("dataset").values(overflow='all')[()].max()
    y_over = histogram.sum("dataset").values(overflow='all')[()][-1]

    import re

    bkg = re.compile('(?!Data)')
    
    background = histogram[bkg]
    data = histogram['Data']

    #fig, ax = plt.subplots(1,1,figsize=(7,7))
    fig, (ax, rax) = plt.subplots(nrows=2,ncols=1, figsize=(7,7),
        gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
    
    # get axes
    hist.plot1d(background, overlay="dataset", ax=ax, stack=True, 
                overflow=bins[binName]['overflow'], clear=False, fill_opts=fillopts2, 
                error_opts=error_opts, order=['DYJets','TTJets', 'ttW', 'ttZ']) #error_opts??
    hist.plot1d(data, overlay="dataset", ax=ax, stack=False, 
                overflow=bins[binName]['overflow'], error_opts=data_err_opts_rat, 
                clear=False)

    hist.plotratio(num=data.sum('dataset'), denom=background.sum('dataset'), ax=rax,
                   error_opts = data_err_opts_rat, denom_fill_opts={}, guide_opts={}, 
                   unc='num', overflow = 'over')

    for l in ['log', 'linear']:
        saveFig(fig, ax, rax, plotDir, name, scale=l, shape=False, y_max=y_max)


In [ ]:
overwrite = True
small = True

tag = 'v0.2.4'


fileset_all_2016   = {'Data': glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/DoubleMuon*2016*/*'),
                'DYJets':glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/DYJetsToLL_M-50_Tune*Summer16*/*.root'),
                'TTJets': glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/TTJets_DiLept_Tune*Summer16*/*'),
                'ttW':glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/TTWJets*Summer16*/*'),
                'ttZ': glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/ttZJets*Summer16*/*')
                 }

fileset_all_2016_sm   = {'Data':glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/DoubleMuon*2016*/*')[:2],
                'DYJets':glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/DYJetsToLL_M-50_Tune*Summer16*/*')[:2],
                'TTJets': glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/TTJets_DiLept_Tune*Summer16*/*')[:2],
                'ttW': glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/TTWJets*Summer16*/*')[:2],
                'ttZ': glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/'+tag+'/ttZJets*Summer16*/*')[:2]
                 }





# load the config and the cache
cfg = loadConfig()

cacheName = 'WH_small' if small else 'WH'

# histograms
histograms = []
histograms += []

# initialize cache
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cfg['caches'][cacheName]), serialized=True)
if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')

else:
    # Run the processor

    if small:
        fileset = fileset_all_2016_sm
        workers = 4
    else:
        fileset = fileset_all_2016
        workers = 16
    
        
    output = processor.run_uproot_job(fileset,
                                      treename='Events',
                                      processor_instance=analysisProcessor(),
                                      executor=exe,
                                      executor_args=exe_args,
                                      chunksize=250000,
                                      #chunksize=100000,
                                     )
    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()